# Análise Comparativa


Essa etapa tem como objetivo comparar alguns modelos para encontrar um que possa ser melhor utilizado dentro do problema, para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

In [235]:
from IPython.display import Markdown
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [236]:
# Ler o csv
df_day = pd.read_csv('day.csv')

# Ler o dicionário de dados
dicionario_dados_day = pd.read_csv('dicionario_dados_day.csv')
df_day.head(1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985


### Tratamento dos dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados. Através do dicionário de dados apresentado no notebook/01_exploratory_data_analysis, é possível perceber que as colunas *casual* e *registered* são deviradas da nossa variável alvo, ou seja, a soma das duas é equivalente ao resultado da coluna **cnt**. Portanto, serão excluídos durante o tratamento e não seguirão para o experimento. A coluna *dteday* também não possui informações relevantes para esta análise.

Foi verificado anteriormente que o dataset não possui valores nulos ou outliers.

Além disso, as colunas categóricas já estão codificadas em sequencias numéricas, enquanto as variáveis contínuas estão calculadas dentro da mesma escala.

In [237]:
df_day2 = df_day.drop(['casual', 'registered', 'dteday'], axis = 1)
df_day2.head(2)

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801


In [238]:
df_day2.columns

Index(['instant', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
       'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'cnt'],
      dtype='object')

In [239]:
# Por padrão, definimos o nome da variável dos previsores como X
X_aluguel = df_day2.drop(columns=['cnt'])

In [240]:
# Definimos o nome da variável com a classe como Y
y_aluguel = df_day2[['cnt']].values

In [219]:
# Fazendo o escalonamento dos valores
scaler_aluguel = StandardScaler()
X_aluguel = scaler_aluguel.fit_transform(X_aluguel)

## Metodologia

Será utilizado o método de validação cruzada **k-fold**. Além disso, serão apresentados os seguintes modelos:

-

-

-

-


## Configuração do experimento

- Separar o conjunto em treino e teste

In [241]:
# X para os atributos previsores e Y para a classe.
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_aluguel, y_aluguel, test_size = 0.3)

In [242]:
X_treinamento.shape, y_treinamento.shape

((511, 12), (511, 1))

In [243]:
X_teste.shape, y_teste.shape

((220, 12), (220, 1))

- Realizar teste com os modelos definidos

## Resultados e discussão

- Expor em uma tabela a comparação dos modelos para explicar qual deles melhor se encaixa para o projeto